In [1]:
import sys
sys.path.insert(0, '/home/christo/notebooks/christo/prod/')
sys.path.insert(0, '/home/christo/notebooks/christo/company_maintenance_scripts/')
import re
import pandas as pd
from NedbankSpendPy import  client
import time
from datetime import datetime 

In [2]:
import numpy as np

In [3]:
import subprocess

In [4]:
from nedbank_companies_addition_lib import maxindexfn, mongo_regex_searchstring
from nedbank_companies_addition_lib import nedbankcompany_class_check, classlookupfn, input_dict_fn, mongo_regex_searchstring_row
from nedbank_companies_addition_lib import  nedbank_companies_db, regex_searchstring

In [7]:
collectionname='aggregated_company_spend'

#### Look at all available extracted_date tags, choose one.

In [5]:
list(client.lookups[collectionname].distinct('extracted_date'))

['20200225', '20200407', '20200408']

#### set the extracted date string here:

In [14]:
extracted_date='20200407'

#### Extract all entries pertaining to THAT extracted date:

In [15]:
df=pd.DataFrame(list(client.lookups[collectionname].find({'extracted_date':extracted_date})))

In [16]:
df.head()

,Seg_L2_Num,Seg_L2_STR,_id,amount_201801,amount_201802,amount_201803,amount_201804,amount_201805,amount_201806,amount_201807,...,n_uniqueclients_201903,n_uniqueclients_201904,n_uniqueclients_201905,n_uniqueclients_201906,n_uniqueclients_201907,n_uniqueclients_201908,n_uniqueclients_201909,n_uniqueclients_201910,n_uniqueclients_201911,n_uniqueclients_201912
0,121.0,Business Banking,5e8c85b50ef767d028613d71,-530.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,32.0,CB - PBCVM - Middle Market,5e8c85b50ef767d028613d72,-10320.7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,61.0,ESTABLISHED,5e8c85b50ef767d028613d73,-1480.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22.0,PB - Gold,5e8c85b50ef767d028613d74,-9919.1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21.0,PB - Platinum,5e8c85b50ef767d028613d75,-5934.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Save the dataframe to a \*.csv file

In [9]:
df.to_csv(path_or_buf='/home/christo/data/aggregated_company_spend_{extracted_date}.csv'.format(extracted_date=extracted_date), index = False)

In [17]:
zip_command = subprocess.run(["zip", 
                             '/home/christo/data/aggregated_company_spend_{extracted_date}.zip'.format(extracted_date=extracted_date),
                            '/home/christo/data/aggregated_company_spend_{extracted_date}.csv'.format(extracted_date=extracted_date)])

#### Create *.zip file to copy to a local drive:

In [18]:
print('zip file name equals: ','/home/christo/data/aggregated_company_spend_{extracted_date}.zip'.format(extracted_date=extracted_date))

zip file name equals:  /home/christo/data/aggregated_franchise_spend_20200407.zip


In [19]:
print(len(df.companyname),len(set(df.companyname)))

118481 303


In [20]:
len(np.where(df.amount_201803.notnull())[0])

4967

In [21]:
cols=list(df)

In [22]:
# n_uniqueclients_201909
client_cols=[]
for col in cols:
#     print(col[:6])
    if col[:6]=='n_uniq':
        client_cols.append(col)

In [23]:
count_cols=[]
for col in cols:
#     print(col[:6])
    if col[:6]=='count_':
        count_cols.append(col)

In [24]:
amount_cols=[]
for col in cols:
#     print(col[:7])
    if col[:7]=='amount_':
        amount_cols.append(col)

In [31]:
count_cols

['count_201801',
 'count_201802',
 'count_201803',
 'count_201804',
 'count_201805',
 'count_201806',
 'count_201807',
 'count_201808',
 'count_201809',
 'count_201810',
 'count_201811',
 'count_201812',
 'count_201901',
 'count_201902',
 'count_201903',
 'count_201904',
 'count_201905',
 'count_201906',
 'count_201907',
 'count_201908',
 'count_201909',
 'count_201910',
 'count_201911',
 'count_201912']

In [33]:
inds=np.where(df[amount_col].notnull())[0]
inds

array([19658, 19659, 19660, ..., 24627, 24628, 24629])

The following is for a strictly company aggregattion:

In [25]:
gf=pd.DataFrame(index=list(set(df.companyname)))
for amount_col,client_col,count_col in zip(amount_cols,client_cols,count_cols):
    print(amount_col,client_col,count_col)
    inds=np.where(df[amount_col].notnull())[0]
    temp_df=df.loc[inds,[amount_col,client_col,count_col, 'companyname']].copy()
    temp_df.set_index('companyname', inplace=True)
    gf=gf.merge(temp_df, left_index=True, right_index=True)

amount_201801 n_uniqueclients_201801 count_201801
amount_201802 n_uniqueclients_201802 count_201802
amount_201803 n_uniqueclients_201803 count_201803
amount_201804 n_uniqueclients_201804 count_201804
amount_201805 n_uniqueclients_201805 count_201805


MemoryError: 

In [81]:
amount_corr_df=pd.DataFrame(index=gf.index)
for amount_now, amount_next in zip(amount_cols[0:-2],amount_cols[1:-1]):
    amount_corr_df[amount_next+'_corr']=np.log(gf[amount_next].div(gf[amount_now], axis=0))

/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until
/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


In [89]:
np.corrcoef(amount_corr_df.values).shape

(26408, 26408)

In [85]:
amount_corr_df.corrwith(other=amount_corr_df,axis=0)

amount_201703_corr    1.0
amount_201704_corr    1.0
amount_201705_corr    1.0
amount_201706_corr    1.0
amount_201707_corr    1.0
amount_201708_corr    1.0
amount_201709_corr    1.0
amount_201710_corr    1.0
amount_201711_corr    1.0
amount_201712_corr    1.0
amount_201801_corr    1.0
amount_201802_corr    1.0
amount_201803_corr    1.0
amount_201804_corr    1.0
amount_201805_corr    1.0
amount_201806_corr    1.0
amount_201807_corr    1.0
amount_201808_corr    1.0
amount_201809_corr    1.0
amount_201810_corr    1.0
amount_201811_corr    1.0
amount_201812_corr    1.0
amount_201901_corr    1.0
amount_201902_corr    1.0
amount_201903_corr    1.0
amount_201904_corr    1.0
amount_201905_corr    1.0
amount_201906_corr    1.0
amount_201907_corr    1.0
amount_201908_corr    1.0
amount_201909_corr    1.0
amount_201910_corr    1.0
amount_201911_corr    1.0
amount_201912_corr    1.0
dtype: float64

In [92]:
def correlation_matrix(df):
    from matplotlib import pyplot as plt
    from matplotlib import cm as cm

    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    cmap = cm.get_cmap('jet', 30)
    C=np.corrcoef(df.values)
    cax = ax1.imshow(C, interpolation="nearest", cmap=cmap)
    ax1.grid(True)
    plt.title('Abalone Feature Correlation')
    labels=list(df)
    ax1.set_xticklabels(labels,fontsize=6)
    ax1.set_yticklabels(labels,fontsize=6)
    # Add colorbar, make sure to specify tick locations to match desired ticklabels
    fig.colorbar(cax, ticks=[.75,.8,.85,.90,.95,1])
    plt.show()

In [93]:
correlation_matrix(df=amount_corr_df)

MemoryError: 

In [82]:
amount_corr_df

,amount_201703_corr,amount_201704_corr,amount_201705_corr,amount_201706_corr,amount_201707_corr,amount_201708_corr,amount_201709_corr,amount_201710_corr,amount_201711_corr,amount_201712_corr,...,amount_201903_corr,amount_201904_corr,amount_201905_corr,amount_201906_corr,amount_201907_corr,amount_201908_corr,amount_201909_corr,amount_201910_corr,amount_201911_corr,amount_201912_corr
*AHK MOTOR SPARES CC,0.567672,0.217895,-0.006666,-0.640660,0.308815,0.153302,-0.235714,-0.276924,0.259297,-0.393654,...,0.233405,-0.363546,0.111399,0.038402,0.440603,-0.774133,-2.153747,0.579312,2.235680,-0.619879
*ASTROJET TRAVEL PTY L,-0.259866,-0.252676,0.229351,-0.275793,-0.185081,0.275019,0.078758,0.418424,0.038935,-0.363494,...,-0.225826,0.276455,0.297404,-0.525642,0.173245,-0.504089,-0.043403,0.492350,-0.371742,NaN
*DR JM JOUBERT,0.733481,-0.234926,0.297357,0.085635,-0.042296,-0.413665,0.214340,0.615446,-0.187723,0.105361,...,-0.184534,0.003317,0.254009,-0.146333,0.013291,0.089291,-0.299076,-0.044397,0.027365,0.432499
*FOREVER NEW P,0.393530,-0.126257,-0.042030,-0.004187,0.255281,-0.226027,0.074049,-0.111702,0.180034,-0.024664,...,0.450721,-0.126402,-0.624325,0.183115,0.190848,0.018058,-0.141708,0.193237,0.059560,0.038198
*FOREVER NEW PTY LTD,0.032624,0.028827,-0.403985,0.284482,0.298211,-0.282810,-0.398622,0.384522,-0.049321,0.186959,...,-0.270180,0.239596,-0.447825,0.028403,0.266813,0.034906,-0.013211,-0.269623,0.103630,0.769874
*GONSA FRUIT AND VEGET,-0.819856,0.641223,-0.917694,0.461674,-0.348583,-0.928596,1.197643,-0.484849,0.360093,0.713301,...,1.110327,-1.013426,0.818497,0.018006,-0.649951,0.449343,0.278225,-0.854508,0.347441,1.107363
*LIQUOR CITY LENASIA,0.442386,-0.127604,0.790621,-1.192697,1.204829,-0.493495,-4.229403,4.228209,-0.161602,0.389074,...,-2.678337,3.072320,-1.226001,-0.601257,-0.184597,1.049132,0.910133,-0.421613,1.142464,0.293540
*LIQUOR CITY PTY LT,0.260268,-0.038900,-0.743052,-0.408321,-0.825509,0.584419,-0.485518,0.150930,1.387660,0.626887,...,0.463008,0.085520,-0.373719,0.237235,-0.337785,0.116681,0.135230,-0.011704,-0.130345,1.557082
*LIQUOR CITY K,-0.166727,-0.257761,0.347576,-0.200396,-0.313658,0.009488,0.021712,-0.067406,-0.032567,0.647128,...,0.460632,0.128460,0.157475,-0.188089,0.117160,0.065876,-0.057854,-0.107235,0.412805,0.188599
*LIQUOR CITY KEY WEST,0.202954,-0.063548,-0.162684,0.085369,-0.240523,0.297942,0.178286,0.221206,-0.175380,0.185797,...,0.123561,-0.013546,0.198631,-0.126551,0.222596,-0.114818,-0.223673,0.059522,0.343277,0.366819


In [75]:
amount_corr_df=pd.DataFrame(index=gf.index)
hf=gf['amount_201704'].div(gf['amount_201703'], axis=0)
amount_corr_df['amount_201704']=hf

In [77]:
amount_corr_df

,amount_201704
*AHK MOTOR SPARES CC,1.243457
*ASTROJET TRAVEL PTY L,0.776719
*DR JM JOUBERT,0.790630
*FOREVER NEW P,0.881389
*FOREVER NEW PTY LTD,1.029246
*GONSA FRUIT AND VEGET,1.898802
*LIQUOR CITY LENASIA,0.880202
*LIQUOR CITY PTY LT,0.961846
*LIQUOR CITY K,0.772780
*LIQUOR CITY KEY WEST,0.938429


In [46]:
inds=np.where(df[amount_col].notnull())[0]

In [48]:
df.loc[inds,[amount_col, 'franchisename']]

,amount_201702,franchisename
0,-81578.40,ON BATH
1,-36783.00,THE CAPITAL ON BATH
2,-193739.50,ABSOLUTE PETS
3,-7514.00,ABSOLUTE PETS (PTY) THE
4,-4297.00,ABSOLUTE PETS - AURO
5,-647.00,ABSOLUTE PETS - AURORA
6,-9930.50,ABSOLUTE PETS - ST J
7,-12785.50,ABSOLUTE PETS - ST J PI
8,-2909.50,ABSOLUTE PETS - ST JOHNS
9,-14595.50,ABSOLUTE PETS BENMOR
